# Implemented a simple recommender system with TensorFlow

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.6.0


Here we have defined users and the list of projects with all the available tags inside the project list.

In [ ]:
users = ['Ajay', 'Yogesh', 'Anirudh', 'Rohan']
projects = ['Weather app', 'Chatbot', 'Object Detector', 'Airline Reservation System', 'TimeTable Generator', 'Space Shooter Game']
tags = ['Android', 'Web Dev', 'TensorFlow', 'Game Dev', 'Python']

The length and number of recommendations to be provided to the users is defined.

In [ ]:
num_users = len(users)
num_proj = len(projects)
num_tags = len(tags)
num_recommendations = 2

A rating matrix is being made where users have rated the project respectively WITH a project matrix which contains tags inside each project.

In [ ]:
#ratings for projects
users_proj = tf.constant([
                            [4,  6,  7,  0, 0, 0],
                            [0,  0, 10,  0, 8, 3],
                            [0,  6,  0,  0, 3, 6],
                            [10, 9,  0,  5, 0, 2]], dtype=tf.float32)

#encoding of the project's tags
proj_tags = tf.constant([
                         [1, 0, 0, 0, 0],
                         [1, 0, 1, 0, 1],
                         [0, 0, 1, 0, 1],
                         [0, 1, 0, 0, 1],
                         [0, 1, 0, 0, 0],
                         [0, 1, 0, 1, 1]], dtype=tf.float32)

A matrix is being mapped between users and the projects


In [ ]:
users_tags = tf.matmul(users_proj,proj_tags)
users_tags

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[10.,  0., 13.,  0., 13.],
       [ 0., 11., 10.,  3., 13.],
       [ 6.,  9.,  6.,  6., 12.],
       [19.,  7.,  9.,  2., 16.]], dtype=float32)>

In [ ]:
#scaling
users_tags = users_tags/tf.reduce_sum(users_tags, axis=1,keepdims=True)
users_tags

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.2777778 , 0.        , 0.3611111 , 0.        , 0.3611111 ],
       [0.        , 0.2972973 , 0.27027026, 0.08108108, 0.35135135],
       [0.15384616, 0.23076923, 0.15384616, 0.15384616, 0.30769232],
       [0.35849056, 0.13207547, 0.16981132, 0.03773585, 0.3018868 ]],
      dtype=float32)>

In [ ]:
#selection the top ones
top_users_tags = tf.nn.top_k(users_tags, num_tags)[1]
top_users_tags

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[2, 4, 0, 1, 3],
       [4, 1, 2, 3, 0],
       [4, 1, 0, 2, 3],
       [0, 4, 2, 1, 3]], dtype=int32)>

In [ ]:
for i in range(num_users):
  tags_names = [tags[int(index)] for index in top_users_tags[i]]
  print('{}: {}'.format(users[i], tags_names))

Ajay: ['TensorFlow', 'Python', 'Android', 'Web Dev', 'Game Dev']
Yogesh: ['Python', 'Web Dev', 'TensorFlow', 'Game Dev', 'Android']
Anirudh: ['Python', 'Web Dev', 'Android', 'TensorFlow', 'Game Dev']
Rohan: ['Android', 'Python', 'TensorFlow', 'Web Dev', 'Game Dev']


Here the ratings is being provided for all the users to the all projects.

In [ ]:
users_ratings = tf.matmul(users_tags, tf.transpose(proj_tags))
users_ratings

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.2777778 , 1.        , 0.7222222 , 0.3611111 , 0.        ,
        0.3611111 ],
       [0.        , 0.6216216 , 0.6216216 , 0.6486486 , 0.2972973 ,
        0.7297298 ],
       [0.15384616, 0.61538464, 0.4615385 , 0.53846157, 0.23076923,
        0.6923077 ],
       [0.35849056, 0.8301887 , 0.4716981 , 0.4339623 , 0.13207547,
        0.4716981 ]], dtype=float32)>

The ratings are removed for those projects users has already checked and rated.

In [ ]:
users_ratings_new = tf.where(tf.equal(users_proj, tf.zeros_like(users_proj)), users_ratings, 
                             tf.zeros_like(tf.cast(users_proj, tf.float32)))
users_ratings_new

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.3611111 , 0.        ,
        0.3611111 ],
       [0.        , 0.6216216 , 0.        , 0.6486486 , 0.        ,
        0.        ],
       [0.15384616, 0.        , 0.4615385 , 0.53846157, 0.        ,
        0.        ],
       [0.        , 0.        , 0.4716981 , 0.        , 0.13207547,
        0.        ]], dtype=float32)>

In [ ]:
top_projects = tf.nn.top_k(users_ratings_new, num_recommendations)[1]
top_projects

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 5],
       [3, 1],
       [3, 2],
       [2, 4]], dtype=int32)>

Final recommended output is provided.

In [ ]:
for i in range(num_users):
  proj_names = [projects[index] for index in top_projects[i]]
  print('{}: {}'.format(users[i],proj_names))

Ajay: ['Airline Reservation System', 'Space Shooter Game']
Yogesh: ['Airline Reservation System', 'Chatbot']
Anirudh: ['Airline Reservation System', 'Object Detector']
Rohan: ['Object Detector', 'TimeTable Generator']
